<a href="https://colab.research.google.com/github/muhuuh/finetuning/blob/main/german_audio_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-z0whk9em/unsloth_01fb655826aa460689db1af2bfdb46ea
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-z0whk9em/unsloth_01fb655826aa460689db1af2bfdb46ea
  Resolved https://github.com/unslothai/unsloth.git to commit e51d3ea2e498fc893770d92ca6727bd113918480
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 136.1 MB/s eta 0:00:00

In [2]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [3]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-2b-it",  # native model for fine-tuning
    #model_name = 'unsloth/Phi-4-mini-instruct-unsloth-bnb-4bit',
    max_seq_length=2048,  # Matches model context; adjust based on your dataset
    dtype=None,  # Auto-detect (bfloat16 if available for better stability)
    load_in_4bit=True  # Enables 4-bit quantization for efficient fine-tuning
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Gemma2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset  # Import this for loading JSONL
import json

# Load the JSONL directly as a Hugging Face Dataset (handles large files efficiently)
ds = load_dataset("json", data_files="./sample_data/german_dictation_cleanup_dataset.jsonl")["train"]

def to_text(ex):
    resp = ex["response"]
    if not isinstance(resp, str):
        resp = json.dumps(resp, ensure_ascii=False)
    msgs = [
        {"role": "user", "content": ex["prompt"]},
        {"role": "assistant", "content": resp},
    ]
    return {
        "text": tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
    }

# Apply mapping: Format into chat template and keep only "text" for SFTTrainer
dataset = ds.map(to_text, remove_columns=ds.column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [5]:
print(dataset[3]["text"])

<bos><start_of_turn>user
wir treffen uns um 14: 00 am Eingang und gehen dann zusammen rein.<end_of_turn>
<start_of_turn>model
Wir treffen uns um 14:00 am Eingang und gehen dann zusammen rein.<end_of_turn>



In [6]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch

# Apply LoRA adapters explicitly for quantized model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Low rank for efficiency
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory-saving during backprop
    random_state=3407,  # For reproducibility (like seed)
    max_seq_length=2048
)

# Now proceed with SFTTrainer as before
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=SFTConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    )
)

trainer.train()

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.4 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1200 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 3 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)


Step,Training Loss
1,5.449700
2,5.624400
3,5.774000
4,5.355900
5,4.766500
6,4.149500
7,3.535000
8,3.195600
9,2.997600
10,2.819800


TrainOutput(global_step=200, training_loss=0.834852976500988, metrics={'train_runtime': 452.8656, 'train_samples_per_second': 7.066, 'train_steps_per_second': 0.442, 'total_flos': 2402702183817216.0, 'train_loss': 0.834852976500988, 'epoch': 2.6666666666666665})

In [7]:
model.save_pretrained_gguf("german_audio_correction_gguf", tokenizer, quantization_method="q4_k_m", maximum_memory_usage = 0.7)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:28<00:00, 88.34s/it]


Unsloth: Merge process complete. Saved to `/content/german_audio_correction_gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['gemma-2-2b-it.F16.gguf

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: All GGUF conversions completed successfully!
Generated files: ['gemma-2-2b-it.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama-cli --model gemma-2-2b-it.Q4_K_M.gguf -p "why is the sky blue?"
Unsloth: Saved Ollama Modelfile to current directory
Unsloth: convert model to ollama format by running - ollama create model_name -f ./Modelfile - inside current directory.


{'save_directory': 'german_audio_correction_gguf',
 'gguf_files': ['gemma-2-2b-it.Q4_K_M.gguf'],
 'modelfile_location': '/content/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': True}

In [8]:
from unsloth import FastLanguageModel

# Enable faster inference mode (after training)
model = FastLanguageModel.for_inference(model)

# Example test prompt (from your dataset/screenshot; user role for correction)
messages = [
    {"role": "user", "content": "ich habe das Ticket erstellt; die die also Nummer ahm genau ist SUP-5555."}
]

# Apply chat template and tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Add assistant start for generation
    return_tensors="pt"
).to("cuda")  # Use GPU if available

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,  # Limit output length
    temperature=0.7,  # Balanced for corrections
    top_p=0.95,
    do_sample=True
)

# Decode and print
response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(response)

# Test another: "Ich glaube dass wir dass bis Sonntag schaffen wenn wir heute anfangen"
messages2 = [{"role": "user", "content": "Ich glaube dass wir dass bis Sonntag schaffen wenn wir heute anfangen"}]
inputs2 = tokenizer.apply_chat_template(messages2, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs2 = model.generate(input_ids=inputs2, max_new_tokens=128, temperature=0.7, top_p=0.95, do_sample=True)
print(tokenizer.decode(outputs2[0][len(inputs2[0]):], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ich habe das Ticket erstellt; die Nummer ist SUP-5555.
Ich glaube, wir schaffen das bis Sonntag, wenn wir heute anfangen.


In [9]:
from huggingface_hub import login, create_repo

# Log in to HF (get token from https://huggingface.co/settings/tokens)
login(token="hf_token")  # Replace with your token

create_repo("ValleHorn/german-audio-correction-gemma2", repo_type="model", exist_ok=True, private=False)



# Save and push LoRA adapters
model.save_pretrained("fine_tuned_adapters")
tokenizer.save_pretrained("fine_tuned_adapters")
model.push_to_hub("ValleHorn/german-audio-correction-gemma2", commit_message="Upload LoRA adapters")

# Merge and save locally first (helps debug)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model")
tokenizer.save_pretrained("merged_model")  # Ensure config.json is there

# Now GGUF push (use local path if needed)
merged_model.push_to_hub_gguf(
    "ValleHorn/german-audio-correction-gemma2",
    tokenizer,
    quantization_method="q4_k_m",
    commit_message="Upload Q4_K_M GGUF"
)

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  50%|#####     | 41.9MB / 83.1MB            

Saved model to https://huggingface.co/ValleHorn/german-audio-correction-gemma2


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...


/usr/local/lib/python3.12/dist-packages/unsloth_zoo/saving_utils.py:969: UserWarning: Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!
  warnings.warn("Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!")


Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...


RuntimeError: Failed to convert model to GGUF: Unsloth: GGUF conversion failed: Unsloth: `config.json` does not exist inside `/tmp/unsloth_gguf_f45ur249`.

In [11]:
print("x")

x


In [13]:
import subprocess
from huggingface_hub import HfApi

# Ensure llama.cpp (clone and build if not found—Colab safe)
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make clean && make -j12  # Build with your Ryzen-like cores; adjust -j

# Save merged model (from previous)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model")
tokenizer.save_pretrained("merged_model")
merged_model.config.save_pretrained("merged_model")  # Ensure config

# Manual GGUF conversion with llama.cpp (use HF-to-GGUF script)
subprocess.run([
    "python", "llama.cpp/convert_hf_to_gguf.py",
    "merged_model",  # Input folder
    "--outfile", "gemma-2-2b-it.Q4_K_M.gguf",  # Output GGUF
    "--outtype", "q4_k_m"  # Quant method
])

# Upload GGUF to HF (adapters already pushed)
api = HfApi()
api.upload_file(
    path_or_fileobj="gemma-2-2b-it.Q4_K_M.gguf",
    path_in_repo="gemma-2-2b-it.Q4_K_M.gguf",
    repo_id="ValleHorn/german-audio-correction-gemma2",
    commit_message="Upload Q4_K_M GGUF for local/Ollama inference"
)

# Optional: Push merged checkpoint too
merged_model.push_to_hub("ValleHorn/german-audio-correction-gemma2", commit_message="Upload merged model")

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  gemma-2-2b-it.Q4_K_M.gguf   :   3%|2         | 50.4MB / 1.71GB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...gsvpf67/model.safetensors:   1%|1         | 28.7MB / 2.22GB            

Saved model to https://huggingface.co/ValleHorn/german-audio-correction-gemma2
